In [1]:
FILE_PATH = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'
IMAGE_SIZE = 56 * 56

In [6]:
from torchvision.datasets import ImageFolder 
from torchvision import transforms 
from torch.utils.data import DataLoader 

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.ToTensor(), 
])

train_dataset = ImageFolder(root=FILE_PATH, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

In [13]:
import torch 
import torch.nn as nn 

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(
                in_channels=1, # Grayscale
                out_channels=16, 
                kernel_size=3
            ), 
            nn.BatchNorm2d(16), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), # max pooling goes after relu activation
            nn.Conv2d(
                in_channels=16, 
                out_channels=32, 
                kernel_size=3
            ),
            nn.BatchNorm2d(32), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                in_features=32*54*54,
                out_features=64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x): 
        x = self.conv_block(x) 
        x = self.fc(x) 

        return x 

In [25]:
import torch.optim as optim 
from tqdm import tqdm 

def train(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Move the model to the selected device
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        total = 0
        correct = 0
        
        # Loop over batches
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            # Move images and labels to the same device as the model
            images = images.to(device)
            labels = labels.to(device).float()  # Convert labels to float
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images).squeeze(1)  # Adjust output dimensions to match labels
            
            # Compute the loss
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            
            # Update weights
            optimizer.step()
            
            # Track loss and accuracy
            running_loss += loss.item()
            preds = (outputs > 0.5).float()  # Convert outputs to binary predictions
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        
        # Print epoch metrics
        epoch_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    print("Training complete!")

In [27]:
model = PneumoniaCNN() 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) 

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, train_loader, criterion, optimizer)

Epoch 1/10:   2%|▏         | 4/163 [00:07<04:50,  1.82s/it]


KeyboardInterrupt: 